<a href="https://colab.research.google.com/github/a0pro0b/Diagnosis-and-Treatment-of-Attention-Deficit-Hyperactivity-Disorder-using-Artificial-Intelligence-/blob/main/ADHD_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path='/gdrive/My Drive/ML/mywork/'

In [ ]:
raw_data=pd.read_csv(path+'allSubs_testSet_phenotypic_dx.csv')
raw_data.head(5)

FileNotFoundError: ignored

ADHD-200 Phenotypic Key


Site

1 Peking University

2 Bradley Hospital/Brown University

3 Kennedy Krieger Institute

4 NeuroIMAGE Sample

5 New York University Child Study Center

6 Oregon Health & Science University

7 University of Pittsburgh

8 Washington University in St. Louis


Gender

0 Female

1 Male


Handedness

0 Left

1 Right

2 Ambidextrous


Diagnosis

0 Typically Developing Children

1 ADHD-Combined

2 ADHD-Hyperactive/Impulsive

3 ADHD-Inattentive


ADHD Measure

1 ADHD Rating Scale IV (ADHD-RS)

2 Conners’ Parent Rating Scale-Revised, Long version (CPRS-LV)

3 Connors’ Rating Scale-3rd Edition


IQ Measure

1 Wechsler Intelligence Scale for Children, Fourth Edition (WISC-IV)

2 Wechsler Abbreviated Scale of Intelligence (WASI)

3 Wechsler Intelligence Scale for Chinese Children-Revised (WISCC-R)

4 Two subtest WASI

5 Two subtest WISC or WAIS – Block Design and Vocabulary


Medication Status

1 Medication Naïve

2 Not Medication Naïve


Quality Control

0 Questionable

1 Pass


PREPROCESSING

DROP UNWANTED FEATURES

In [ ]:
raw_data=raw_data.drop(columns=['Disclaimer','ID','Site','Secondary Dx ','Full2 IQ','QC_Rest_1','QC_Rest_2','QC_Rest_3','QC_Rest_4','QC_Anatomical_2'])

In [ ]:
raw_data

FILL NAN BY 0

In [ ]:
raw_data=raw_data.fillna(0)

In [ ]:
raw_data

DELETING PENDING ROWS

In [ ]:
dropset=raw_data.index[raw_data['DX']=="pending"].tolist()
dropset,len(dropset)

In [ ]:
raw_data.drop(raw_data.index[dropset],inplace=True)

In [ ]:
raw_data

In [ ]:
raw_data=raw_data.reset_index()
raw_data

SPLITTING IP AND OP

In [ ]:
ip=raw_data.drop(columns=['DX'])

In [ ]:
ip.shape

In [ ]:
ip

In [ ]:
op=raw_data['DX']

In [ ]:
op.shape

In [ ]:
op

WHEN WE SPLIT TRAIN AND TEST DATA THERE IS A CHANCE OF CLASS IMBALANCE AS DX:2 APPEARS IN LESS SAMPLES

SO y_train or y_test MIGHT NOT HAVE DX:2

SO WE INCREASE THE SAMPLES USING SMOTE

WHEN USING SMOTE k_neighbours is set to 1 as DX:2 has only 2 samples, so k_neighbour can take a value < 2


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(k_neighbors=1,random_state=100)
#ip,op=sm.fit_sample(ip,op.ravel())
#sm.fit(x_test,y_test)
ip,op=sm.fit_resample(ip,op.ravel())

In [ ]:
ip.shape,op.shape

NOTE ABOVE THE NUMBER OF SAMPLES IS INCREASED FROM 171 TO 376

SPLIT TEST AND TRAIN

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(ip,op,test_size=0.25, random_state=2)
one=x_train
two=x_test
three=y_train
four=y_test

In [ ]:
l=np.unique(y_train)
l

In [ ]:
ll=np.unique(y_test)
ll

In [ ]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

APPLYING THE MODELS

APPLY LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
fold=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
fold.get_n_splits(x_train,y_train)

In [ ]:
val=list(np.arange(0.001,1,0.1))

In [ ]:
parameters={"solver":("newton-cg","liblinear"),"C":val}

In [ ]:
model=LogisticRegression(penalty='l2',n_jobs=-2,max_iter=1000)
log_res=GridSearchCV(model,parameters,n_jobs=-2,cv=fold)

In [ ]:
log_res.fit(x_train,y_train)

In [ ]:
print(log_res.best_params_)

In [ ]:
y_pred_log_res=log_res.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred_log_res))
ac=accuracy_score(y_test,y_pred_log_res)
print(ac*100)

APPLY KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.metrics import fbeta_score,make_scorer
#fscorer=make_scorer(fbeta_score,beta=1)
params={'n_neighbors':range(3,10)}
knn=KNeighborsClassifier(algorithm='brute')
knn_grid=GridSearchCV(knn,params,cv=fold,n_jobs=-2)
knn_grid.fit(x_train,y_train)

In [ ]:
print(knn_grid.best_params_)

In [ ]:
y_pred_knn_grid=knn_grid.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred_knn_grid))
ac=accuracy_score(y_test,y_pred_knn_grid)
print(ac*100)

APPLY BERNOULLI/GUASSIAN NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import BernoulliNB
NB_Comp=BernoulliNB()
NB_Comp.fit(x_train,y_train)
y_pred_NBComp=NB_Comp.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred_NBComp))
ac=accuracy_score(y_test,y_pred_NBComp)
print(ac*100)

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB_Comp=GaussianNB()
NB_Comp.fit(x_train,y_train)
y_pred_NBComp=NB_Comp.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred_NBComp))
ac=accuracy_score(y_test,y_pred_NBComp)
print(ac*100)

APPLY DECISION TREE MODEL

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree=DecisionTreeClassifier(criterion='gini')
dtree.fit(x_train,y_train)
y_pred_dt=dtree.predict(x_test)
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_pred_dt))
ac=accuracy_score(y_test,y_pred_dt)
print(ac*100)

In [ ]:
path=dtree.cost_complexity_pruning_path(x_train,y_train)

In [ ]:
alpha=path.ccp_alphas
print(alpha)

In [ ]:
impurities=path.impurities

In [ ]:
new_trees=[]
i=0
for a in alpha:
  new_tree=DecisionTreeClassifier(criterion='gini',ccp_alpha=a)
  new_tree.fit(x_train,y_train)
  new_trees.append(new_tree)
  i=i+1


In [ ]:
test_scores=[new_tree.score(x_test,y_test) for new_tree in new_trees]
print(test_scores)
print('max f1 test score={}'.format(max(test_scores)))
print('position of max test scores{}'.format(np.argmax(test_scores)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
prunedtree=DecisionTreeClassifier(criterion='gini',ccp_alpha=alpha[np.argmax(test_scores)])
prunedtree.fit(x_train,y_train)
new_y_pred_train=prunedtree.predict(x_train)
new_y_pred_test=prunedtree.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,new_y_pred_test))
ac=accuracy_score(y_test,new_y_pred_test)
print(ac*100)

APPLY SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC(C=1.0, random_state=1, kernel='linear')
svc.fit(x_train, y_train)
y_predict = svc.predict(x_test)


In [ ]:
from sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_test,y_predict))
ac=accuracy_score(y_test,y_predict)
print(ac*100)

TO IMPROVE ACCURACY WE COMBINE MODELS AND DO ENSEMBLING

APPLY RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)

In [ ]:
y_pred_rf= rf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rf))

APPLY LOGISTIC REGRESSION WITH RANDOM FOREST

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# It is important to train the ensemble of trees on a different subset of the training 
#data than the linear regression model to avoid overfitting, in particular 
#if the total number of leaves is similar to the number of training samples
x_train, X_train_lr, ytrain, y_train_lr = train_test_split(x_train, y_train, test_size=0.60, shuffle = True)

rf = RandomForestClassifier(n_estimators=1000)
#train Random forest on a set of samples
rf.fit(x_train, ytrain)

#one hot encoder of the leaves
rf_enc = OneHotEncoder()

#for these leaves of RF, fit the one hot encoder
rf_enc.fit(rf.apply(x_train))

In [ ]:
f = rf_enc.transform(rf.apply(X_train_lr))
f.shape

In [ ]:
fold = StratifiedKFold(n_splits=5, random_state=1, shuffle = True)
fold.get_n_splits(X_train_lr, y_train_lr)

parameters = {'solver':('liblinear', 'sag'), 'C':[0.001]}

model = LogisticRegression(penalty='l2', n_jobs = -2, max_iter=1000)
rf_lr = GridSearchCV(model, parameters, n_jobs= -2, cv=fold)

rf_lr.fit(rf_enc.transform(rf.apply(X_train_lr)), y_train_lr)

print(rf_lr.best_params_)

In [ ]:
y_pred_rf= rf_lr.predict(rf_enc.transform(rf.apply(x_test)))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rf))

APPLY RANDOM FOREST ON KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knnmodel=KNeighborsClassifier(algorithm='brute')
params={'n_neighbors':range(3,10)}
knn_grid=GridSearchCV(knnmodel,params,cv=fold,n_jobs=-2)
knn_grid.fit(rf_enc.transform(rf.apply(X_train_lr)), y_train_lr)
print(knn_grid.best_params_)

In [ ]:
y_pred_knn= knn_grid.predict(rf_enc.transform(rf.apply(x_test)))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_knn))

APPLY NEURAL NETWORK

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop, Adamax, Adagrad, Nadam
import keras.backend as K

In [ ]:
model = Sequential()
model.add(Dense(30, input_shape = (14,), activation = "relu"))
model.add(Dense(10, activation = "relu"))
model.add(Dense(4, activation = "softmax"))
model.compile(optimizer='Nadam',   metrics = ["accuracy"], loss='categorical_crossentropy')
model.summary()

In [ ]:
from keras.utils.np_utils import to_categorical
y_cat = to_categorical(op)
model.fit(x=ip, y=y_cat, verbose=1, epochs=200)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(two)
y_test_class = np.argmax(y_pred,axis=1)
l=[]
for i in range(0,94):
  l.append(int(four[i]))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, l))